In [2]:
from omegaconf import DictConfig, OmegaConf
import hydra
import dataload
import embed
import evaluate
from utils.data_utils import data_to_kg, extract_ents
import preprocess
from sklearn.metrics import accuracy_score, confusion_matrix
import fire
import sys
import tqdm
import pandas as pd
import numpy as np
import kgbench as kg
from collections import Counter
import matplotlib.pyplot as pl
import random as rd
from operator import itemgetter
import torch
import torch.nn.functional as F
from utils import RDF_NUMBER_TYPES, get_relevant_relations, add_triple, get_p_types
from kgbench.load import Data
from typing import List, Sequence, Tuple

from kgbench import load, tic, toc, d
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from utils import URI_PREFIX
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from kgbench import load
import dgl

c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = kg.load(name="mdgenre", torch=True, final=True, prune_dist=2 )

loaded data mdgenre (80.59s).
pruned (14.33s).


In [25]:
data.num_classes

12

In [8]:
dgl_df = data.dgl()


In [26]:
g = dgl_df[0]

In [30]:
g.ndata['training_mask']

{'training_mask': tensor([False, False, False,  ..., False, False, False]), 'withheld_mask': tensor([False, False, False,  ..., False, False, False]), 'label': tensor([-1, -1, -1,  ..., -1, -1, -1])}

In [29]:

len(g.nodes())

331089

In [26]:
#dataset = kg.load(name="mdgenre", torch=True, final=True, prune_dist=2 ).dgl(to32=True)

g = dataset[0].int()



num_rels = len(g.canonical_etypes)
num_classes = dataset.num_classes

print(f'Loaded {"mdgenre"}: {num_rels} relations, {num_classes} classes.')

category = dataset.predict_category
labels = g.nodes[category].data.pop('label') # note: not labels

training_mask = g.nodes[category].data.pop('training_mask')
withheld_mask = g.nodes[category].data.pop('withheld_mask')
    # -- note the use of training/withheld rather than train/test

for cetype in g.canonical_etypes:
    g.edges[cetype].data['norm'] = dgl.norm_by_dst(g, cetype).unsqueeze(1) * 0.0 + 1.0

g = dgl.to_homogeneous(g, edata=['norm'])

training_idx = training_mask.nonzero()
withheld_idx = withheld_mask.nonzero()
    # -- nonzero() produces the indices of all the elements that are true

training_labels = labels[training_idx].squeeze()
withheld_labels = labels[withheld_idx].squeeze()

Loaded mdgenre: 154 relations, 12 classes.


KeyError: 'label'

In [20]:
category

'resource'

In [13]:
from dgl.nn import TransR,TransE
model = TransR(num_rels=num_rels, rfeats=1,nfeats=1)

In [17]:
data.triples[:,0]

tensor([176413, 173816, 197157,  ..., 180476, 198138, 158250],
       dtype=torch.int32)

In [14]:
model.forward(data.triples[:,0],data.triples[:,2],data.triples[:,1])

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 6076658847744 bytes.

In [ ]:
model = RGCN(num_nodes=g.num_nodes(), in_dim=256, h_dim=16, out_dim=dataset.num_classes, num_rels=num_rels, num_bases=40, reg='basis')


optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

model.train()
for epoch in range(50):
    logits = model(g)[training_idx].squeeze(1)
    loss = F.cross_entropy(logits, training_labels.to(torch.long))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    acc = (logits.argmax(dim=1) == training_labels).sum() / training_labels.size(0)
    print(f'Epoch {epoch:05d} | Loss {loss.item():.4f} | Train Accuracy {acc:.4f} ')

    acc = evaluate(g, withheld_idx, withheld_labels, model)
    print("         Test accuracy {:.4f}".format(acc))

In [36]:
from dgl.nn import TransR,TransE
scorer = TransE(num_rels=num_rels, feats=f)


TypeError: __init__() got an unexpected keyword argument 'num_nodes'

In [15]:
from torch import nn
from dgl.nn.pytorch import RelGraphConv

In [16]:
class RGCN(nn.Module):
    """
    NB: This is an _embedding_ RGCN. It's slightly different from the classic RGCN in rgcn.py.
    """

    def __init__(self, num_nodes, in_dim, h_dim, out_dim, num_rels, num_bases, reg):
        reg = None if reg == 'none' else reg

        super().__init__()

        self.emb = nn.Embedding(num_nodes, in_dim)
        # nn.init.kaiming_normal_(self.emb.weight, mode='fan_in')

        # two-layer RGCN
        self.conv1 = RelGraphConv(in_dim, h_dim, num_rels, regularizer=reg, num_bases=num_bases, self_loop=False)
        self.conv2 = RelGraphConv(h_dim, out_dim, num_rels, regularizer=reg, num_bases=num_bases, self_loop=False)

    def forward(self, g):

        x = self.emb.weight

        h = F.relu(self.conv1(g, x, g.edata[dgl.ETYPE], g.edata['norm']))

        h = self.conv2(g, h, g.edata[dgl.ETYPE], g.edata['norm'])

        return h

def evaluate(g, withheld_idx, labels, model):
    model.eval()
    with torch.no_grad():
        logits = model(g)[withheld_idx].squeeze(1)

    acc = (logits.argmax(dim=1) == labels).sum() / labels.size(0)

    return acc.item()

In [17]:
model = RGCN(num_nodes=g.num_nodes(), in_dim=256, h_dim=16, out_dim=dataset.num_classes, num_rels=num_rels, num_bases=40, reg='basis')


optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)

model.train()
for epoch in range(50):
    logits = model(g)[training_idx].squeeze(1)
    loss = F.cross_entropy(logits, training_labels.to(torch.long))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    acc = (logits.argmax(dim=1) == training_labels).sum() / training_labels.size(0)
    print(f'Epoch {epoch:05d} | Loss {loss.item():.4f} | Train Accuracy {acc:.4f} ')

    acc = evaluate(g, withheld_idx, withheld_labels, model)
    print("         Test accuracy {:.4f}".format(acc))

Epoch 00000 | Loss 2.4662 | Train Accuracy 0.0280 
         Test accuracy 0.5410
Epoch 00001 | Loss 2.3675 | Train Accuracy 0.5449 
         Test accuracy 0.2080
Epoch 00002 | Loss 2.3304 | Train Accuracy 0.2178 
         Test accuracy 0.2093
Epoch 00003 | Loss 2.2975 | Train Accuracy 0.2221 
         Test accuracy 0.2097
Epoch 00004 | Loss 2.2796 | Train Accuracy 0.2219 
         Test accuracy 0.2093
Epoch 00005 | Loss 2.2679 | Train Accuracy 0.2241 
         Test accuracy 0.2070
Epoch 00006 | Loss 2.2498 | Train Accuracy 0.2262 
         Test accuracy 0.2100
Epoch 00007 | Loss 2.2394 | Train Accuracy 0.2266 
         Test accuracy 0.2113


KeyboardInterrupt: 

In [20]:
from dgl.nn import TransR,TransE


In [19]:
scorer = TransE(num_rels=len(data.i2r), feats=1)


In [21]:
scorer.forward()

TypeError: forward() missing 3 required positional arguments: 'h_head', 'h_tail', and 'rels'

In [38]:
import graphembedding
from graphembedding import TransE
a = TransE(data.triples)

ModuleNotFoundError: No module named 'graphembedding'

In [1]:
import graphembedding

In [2]:
from graphembedding import TransE
a = TransE(data.triples)

ImportError: cannot import name 'TransE' from 'graphembedding' (c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\graphembedding-0.1-py3.8.egg\graphembedding\__init__.py)

In [9]:
from graphembedding import complEx

In [10]:
a = complEx(data.triples)

KeyError: 185557

In [11]:
from graphembedding.playground import load_github
github_dataset = load_github() 
triplets = github_dataset[['subject','relation','object']].values

Exception: URL fetch failure on https://storage.googleapis.com/github-playground/playground.h5: 403 -- Forbidden

In [4]:
import neuralkg
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
import wandb
from neuralkg.utils import setup_parser
from neuralkg.utils.tools import *
from neuralkg.data.Sampler import *
import yaml

In [18]:
import os
os.listdir()

['dataload',
 'embed',
 'evaluate',
 'main.py',
 'outlier-detection.ipynb',
 'outputs',
 'preprocess',
 'process.py',
 'test.png',
 'test.py',
 'train_model.py',
 'transE.ipynb',
 'typings',
 'unit_test.ipynb',
 'utils',
 '__init__.py']

In [4]:
parser = setup_parser()
args = parser.parse_args(args=[])
#specify the config path
config_path = "../config/TransE_demo_kg.yaml"

#load config to args
args = load_config(args, config_path)
args.data_path = "../dataset/FB15K237"
#set random seeds
seed_everything(args.seed)
#set up sampler,datamodule,model,litmodel dynamicly
train_sampler = import_class(f"neuralkg.data.{args.train_sampler_class}")(args)
test_sampler = import_class(f"neuralkg.data.{args.test_sampler_class}")(train_sampler)
kgdata = import_class(f"neuralkg.data.{args.data_class}")(args, train_sampler, test_sampler)
model = import_class(f"neuralkg.model.{args.model_name}")(args)
lit_model = import_class(f"neuralkg.lit_model.{args.litmodel_name}")(model, args)
#set up logger, TensorBoardLogger is used by default
logger = pl.loggers.TensorBoardLogger("training/logs")
if args.use_wandb:
    log_name = "_".join([args.model_name, args.dataset_name, str(args.lr)])
    logger = pl.loggers.WandbLogger(name=log_name, project="NeuralKG")
    logger.log_hyperparams(vars(args))
#set up early_callback to early stopping
early_callback = pl.callbacks.EarlyStopping(
        monitor="Eval|mrr",
        mode="max",
        patience=args.early_stop_patience,
        check_on_train_epoch_end=False,
    )
#set up model saving method
dirpath = "/".join(["output", args.eval_task, args.dataset_name, args.model_name])
model_checkpoint = pl.callbacks.ModelCheckpoint(
    monitor="Eval|mrr",
    mode="max",
    filename="{epoch}-{Eval|mrr:.3f}",
    dirpath=dirpath,
    save_weights_only=True,
    save_top_k=1,
)
callbacks = [early_callback, model_checkpoint]
# initialize trainer
trainer = pl.Trainer.from_argparse_args(
        args,
        callbacks=callbacks,
        logger=logger,
        default_root_dir="training/logs",
        gpus="0,",
        check_val_every_n_epoch=args.check_per_epoch,
    )


Global seed set to 321
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [5]:
#train&eval model
trainer.fit(lit_model, datamodule=kgdata)
#test model
lit_model.eval()
trainer.test(lit_model, datamodule=kgdata)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: training/logs\default

  | Name  | Type     | Params
-----------------------------------
0 | model | TransE   | 443 K 
1 | loss  | Adv_Loss | 443 K 
-----------------------------------
443 K     Trainable params
2         Non-trainable params
443 K     Total params
1.773     Total estimated model params size (MB)


Global seed set to 321


Epoch 0:   0%|          | 10/54423 [00:52<79:29:55,  5.26s/it, loss=0.856, v_num=0]

c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\pytorch_lightning\utilities\data.py:59: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 10. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 1: 100%|██████████| 71958/71958 [26:15<00:00, 45.68it/s, loss=0.714, v_num=0, Eval|mrr=0.007, Eval|hits@10=0.009]

OSError: [Errno 22] Invalid argument: 'E:/00_CODE/03_Master_Thesis/rdf-literal-preprocessing/src/output/link_prediction/demo_kg/TransE/epoch=1-Eval|mrr=0.007.ckpt'

Epoch 1: 100%|██████████| 71958/71958 [26:23<00:00, 45.44it/s, loss=0.714, v_num=0, Eval|mrr=0.007, Eval|hits@10=0.009]

In [30]:
import torch
import torch.nn as nn

class TransE(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        super(TransE, self).__init__()
        self.entity_embeddings = nn.Embedding(num_entities, embedding_dim)
        self.relation_embeddings = nn.Embedding(num_relations, embedding_dim)
    def forward(self, head, relation, tail):
        head_embedding = self.entity_embeddings(head)
        relation_embedding = self.relation_embeddings(relation)
        tail_embedding = self.entity_embeddings(tail)
        score = torch.norm(head_embedding + relation_embedding - tail_embedding, p=1, dim=1)
        return score

In [67]:
model = TransE(data.num_entities,data.num_relations,1)

In [69]:
model(data.triples[:,0],data.triples[:,1],data.triples[:,2])

tensor([3.8390, 0.7453, 0.4912,  ..., 0.6131, 0.4018, 1.4004],
       grad_fn=<NormBackward1>)

In [34]:
for i, (head, relation, tail) in enumerate(data.triples):
    print(head, relation, tail)
    break

tensor(176413, dtype=torch.int32) tensor(131, dtype=torch.int32) tensor(120236, dtype=torch.int32)


In [57]:
def get_neg_sample(data,head,relation):
    df = data.triples
    ents = torch.arange(data.num_entities)
    sub = df[df[:,0] == head ]
    #print(sub)
    sub = sub[sub[:,1] == relation ]
    print(sub[:,2])
    #print(df[df[:,0] == head ][:,2])

    positive_ents = torch.unique(sub[:,2])
    print(len(ents))
    for e in positive_ents:
        ents = torch.cat((ents[:e], ents[e+1:]))
        #ents=ents[ents[~e]]
    print(len(ents))

    random_index = torch.randint(0, len(ents), (1,))

    # Access the random element in the tensor
    return ents[random_index]
    
    #df[df[:,0]== head and df[:,0]== relation][]

In [59]:
print(data.triples[0,2])
get_neg_sample(data,data.triples[0,0],data.triples[0,1])

tensor(120236, dtype=torch.int32)
tensor([120236], dtype=torch.int32)
331089
331088


tensor([240154])

In [1]:
num_entities = data.num_entities
num_relations = data.num_relations
model = TransE(num_entities, num_relations, 1)
model(data.triples[:2,0],data.triples[:2,1],data.triples[:2,2])

NameError: name 'data' is not defined

In [61]:
import torch.optim as optim
num_entities = data.num_entities
num_relations = data.num_relations
embedding_dim = 1
num_epochs = 20
# Define model, optimizer, and loss function
model = TransE(num_entities, num_relations, embedding_dim)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MarginRankingLoss(margin=1.0)

# Training loop
for epoch in range(num_epochs):
    for i, (head, relation, tail) in enumerate(data.triples):
        optimizer.zero_grad()
        # Generate negative samples
        #negative_tail = torch.randint(num_entities, size=(len(head),))
        # Forward pass
        positive_score = model(head, relation, tail)
        negative_score = model(head, relation, get_neg_sample(data,head,relation))
        # Compute loss
        loss = loss_fn(positive_score, negative_score, torch.ones(len(head)))
        loss.backward()
        optimizer.step()
    print("Epoch {}/{}, Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [5]:
from graphembedding import complEx, transE

In [6]:
transE(data.triples)

KeyError: 176413

In [7]:
from pykeen.pipeline import pipeline

result = pipeline(
    model='TransE',
    dataset='nations',
)

No random seed is specified. Setting to 1173960838.
Training epochs on cuda:0: 100%|██████████| 5/5 [00:11<00:00,  2.37s/epoch, loss=1.41, prev_loss=1.39]
INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=201.
Evaluating on cuda:0: 100%|██████████| 201/201 [00:00<00:00, 4.06ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.07s seconds


In [8]:
result

PipelineResult(random_seed=1173960838, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(14, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(55, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=14, num_relations=55, create_inverse_triples=False, num_triples=1592, path="C:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\Lib\site-packages\pykeen\datasets\nations\train.txt"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x00000184546FDDC0>, losses=[1.4652120215552193, 1.4338116134916032, 1.4517410823277064, 1.3901010070528304, 1.4133971759251185], metric_results=<pykeen.evaluation.rank_based_evaluator.RankBasedMetricResults object at 0x00000184546B1400>, train_seconds=17.051034927368164, evaluate_seco